In [1]:
from VAE_model import build_model, pack, unpack
import glob
import numpy as np
from tqdm import tqdm
from lucid.misc.io import load
from lucid.optvis.param import cppn
from lucid.optvis import objectives
from lucid.optvis import render
from lucid.misc.tfutil import create_session
import tensorflow as tf
from lucid.modelzoo import vision_models
import pylab as plt

/home/hoppeta/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/hoppeta/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
cutoff = 128**10
image_size = 400
size_n = 200
F_MODELS = glob.glob("results/VAE_base_models/*.npy")[:cutoff]

In [ ]:
# Load the save model params
params0 = [load(f_model) for f_model in tqdm(F_MODELS)]
shapes = list(map(lambda x:x.shape, params0[0]))
X = np.array([unpack(p) for p in params0])

 10%|▉         | 101/1024 [00:01<00:09, 98.24it/s]

In [4]:
class render_model():
    def __init__(self):

        print ("Loading Inception model")
        self.model = vision_models.InceptionV1()
        self.model.load_graphdef()

        sess = create_session()

        self.t_size = tf.placeholder_with_default(size_n, [])
        obj = objectives.channel("mixed4a", 0)
        self.T = render.make_vis_T(
            self.model, obj,
            param_f=lambda: cppn(self.t_size),
        )

        tf.global_variables_initializer().run()
        self.train_vars = sess.graph.get_collection(
            tf.GraphKeys.TRAINABLE_VARIABLES)

    def __call__(self, params):

        feed_dict = dict(zip(self.train_vars, params))
        feed_dict[self.t_size] = image_size
        return self.T("input").eval(feed_dict)[0]


In [5]:
latent_n = 2
intermediate_n = 512
input_n = 8451
f_h5 = 'results/VAE_weights.h5'

VAE, encoder, decoder = build_model(
    input_n, intermediate_n, latent_n)
VAE = VAE.load_weights(f_h5)

z_mean, z_std_log, z = encoder.predict(X, batch_size=128)
xp = decoder.predict(z)
print (z_mean)

[[-0.1148695   0.01965168]
 [-0.075947   -0.0616593 ]
 [ 0.03413891 -0.0824635 ]
 ...
 [ 0.0253809  -0.03752047]
 [ 0.05214637 -0.0361219 ]
 [ 0.0103711  -0.04150574]]


In [6]:
params1 = [pack(x, shapes) for x in decoder.predict(z)]

In [7]:
R = render_model()

Loading Inception model


In [10]:
params0[3]

array([-1.0501842 ,  0.43053436, -0.9149284 , ...,  0.06743582,
       -0.1828395 , -0.10006539], dtype=float32)